In [1]:
# NOTE: Work-in-progress revision to testing paradigm.
#
# This directory contains alias tests using the BatsPP testing infrastructure framework.
# (Bash functions are included as well, so "alias" is used in a generic sense. We also
# refer to them collectively as "macros", but that is not a common usage elsewhere.
#
# The tests can be defined via Jupyter notebooks or ascii shell-script like files with output
# included (.e.e., the .batspp format). 
#     https://github.com/LimaBD/batspp
# The get converter to Bats tests:
#     https://github.com/bats-core/bats-core
#
# The aliases to be defined are taken from 
#      aliases-for-testing.bash
# Currently, this in turn gets the alias and functions from the following files:
#      tomohara-aliases.bash, more-tomohara-aliases.bash, tomohara-proper-aliases.bash
# For historical reasons, most of the alias files involve 'tomohara'.
#
# The Bash kernel is used in order to evaluate notebooks under Jupyter:
#      https://github.com/takluyver/bash_kernel
# Currently, aliases-for-testing.bash needs to be sourced from your ~/.bashrc file for
# the aliases to be defined.
#--------------------------------------------------------------------------------------
# Warnings
#
# - *** Run under a less privileged testing account (e.g., testuser) to avoid 
# inadvertant loss of data ***. Some of the tests delete temporary folders which might
# be inadvertantly include useful files. In additon, they might make configuration changes.
#
# - When starting Juptyter, the best way to ensure the right macros are loaded is follows:
#      cd .../shell-scripts/tests
#      BASH_ENV="$PWD/aliases-for-testing.bash" run-jupyter-notebook
# This way the Bash kernel will use the right set in case you have multiple repo's.
# (Otherwise, you need to modify ~/.bashrc to conditionally load the desired set.)
#--------------------------------------------------------------------------------------
# General Tips
#
# *** Remember to "Keep It Simple, Sam" (i.e., KISS principle). That is, test for simple 
# things that can be verified with grep or similar commands. For example, rather than
# outputting an entire pylint report, grep for specific issues (e.g., redefining keyword).
#
# For simplicity, the output can still be done as before. However, we will need to
# add a mechanism so that the test target doesn't change to match the alias revision.
# For example, if alias fubar were changed to output "foobar", the test output would
# change once the Jupyter notebook output gets re-generated.
#
# We will extend bats_pp to allow for special case tests of the following format:
#    condition; echo $?
# The test will be considered passing if and only if the output is 0.
# For example,
#    num_lines=$(my-alias | grep -c "pattern"}
#    [ $num_lines -gt 2 ]; echo $?
#
# Therefore, one cell can show the command output as done before, and the next cell
# show the test. For example,
#    cell 1:
#       code: 
#           echo "a b c D E F" > _abcDEF_.txt_
#           cat _abcDEF_.txt_
#       output:
#           a b c D E F
#    cell 2:
#       code:
#           num_lower=$(tr " " "\n" _abcDEF_.txt_ | grep -c "[a-z]")
#           [ $num_lower -eq 3 ]; echo $?
#       output:
#           0
#

In [2]:
## DEBUG
## # Show user groups
## (whoami; groups)

In [3]:
# Warn if user not in operators group
is_operator=$(groups | grep -c operator)
if [ $is_operator -eq 0 ]; then
    echo "*** Warning: should be run as operator ***" 1>&2
fi

In [5]:
# Abort if user in admin-like group unless FORCE_RUN env var set
# Note: The bash kernel restarts, so this just blocks run-all-cells usage.
# TODO1: * add script for use here and in batspp_report.py!
force_run="${FORCE_RUN:-0}"
is_admin=$(groups $USER | egrep -c "root|admin|adm")
jupyter_session=${JPY_SESSION_NAME:-""}
## DEBUG: trace-vars force_run is_admin jupyter_session
if [ $is_admin -eq 1 ]; then
    if [ "$force_run" == "0" ]; then
        echo "*** Error: should not be run as operator unless FORCE_RUN ***" 1>&2
        if [ "$jupyter_session" != "" ]; then
            exit
        fi
    fi
    echo "FYI: not recommended to run under admin account"
fi

In [6]:
# Global setup
#
# NOTE: For reproducibility, the directory name needs to be fixed
# In place of $$, use a pseudo-random number (e,g., 1210)
# *** All output from one run to the next needs to be the same ***

# Set a temp directory for this test (should be unique)
TMP=${TMP:-/tmp}
if [ "$DEBUG_LEVEL" -lt 4 ]; then TMP=/tmp/test-tips; fi
#
# note: if in doubt for a good random number, try the following:
#    python -c "import random; print(random.randint(100000, 999999))"
temp_dir=$TMP/657398

# Create the directory and changed into it
# Note: uses command to avoid escapes for cd/pwd aliases
mkdir -p "$temp_dir"
command cd "$temp_dir"
command pwd

/tmp/test-tips/657398


In [7]:
# TEMP: 
# To avoid issues with 'rm -r' the file clobber operator is used (i.e., ">|")
# That itself is dangerous, so the filenames start and end with _ to minimize conflict.
# For example, "file.txt" => "_file.txt_").

In [8]:
# Create test file with letters
echo "a b c D E F" >| _abcDEF_.txt_
cat _abcDEF_.txt_

a b c D E F


In [9]:
# Make sure 3 lowercase chars
# note: count-it is alias tabulating regex pattern occurrences
num_lower=$(count-it "[a-z]" _abcDEF_.txt_ | wc -l)
[ $num_lower -eq 3 ]; echo $?

0
